# 01 - Exploration des Donnees Electorales - Schema v3.0

**Projet :** Electio-Analytics - Prediction Presidentielles 2027  
**Schema :** v3.0 (17 tables, systeme polymorphe de territoire)  
**Perimetre :** 534 communes de Gironde (departement 33)  
**Elections :** Presidentielles 2017 & 2022 (T1 + T2)  

---

## Objectifs

- Verifier les donnees chargees en base (volumes, coherence)
- Explorer les 534 communes (population, superficie)
- Analyser participation et resultats par candidat
- Comparer 2017 vs 2022 pour les 7 candidats communs
- Visualiser les indicateurs securite Bordeaux (2016-2024)

**Note technique :** Les `id_territoire` des resultats electoraux utilisent le format `'33' + code_insee` (7 chars),
tandis que `commune.id_commune` utilise le code INSEE brut (5 chars). On normalise via `id_territoire[2:]`.

In [17]:
# Imports + Connexion DB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

import sys
sys.path.insert(0, '..')

from sqlalchemy import func
from src.database.config import get_engine, get_session
from src.database.models import (
    Region, Departement, Commune, Canton,
    TypeElection, Election, Candidat,
    ElectionTerritoire, ResultatParticipation, ResultatCandidat,
    TypeIndicateur, Indicateur, Prediction
)

engine = get_engine()
session = get_session()
print("Connexion DB OK")

Connexion DB OK


In [18]:
# Verification tables - COUNT(*) via ORM
models_to_check = [
    ('region', Region),
    ('departement', Departement),
    ('commune', Commune),
    ('canton', Canton),
    ('type_election', TypeElection),
    ('election', Election),
    ('candidat', Candidat),
    ('election_territoire', ElectionTerritoire),
    ('resultat_participation', ResultatParticipation),
    ('resultat_candidat', ResultatCandidat),
    ('type_indicateur', TypeIndicateur),
    ('indicateur', Indicateur),
    ('prediction', Prediction),
]

counts = []
for name, model in models_to_check:
    n = session.query(func.count()).select_from(model).scalar()
    counts.append({'table': name, 'lignes': n})

df_counts = pd.DataFrame(counts)
print(df_counts.to_string(index=False))
print(f"\nTotal : {df_counts['lignes'].sum()} lignes")

                 table  lignes
                region       1
           departement       1
               commune     534
                canton       0
         type_election       1
              election       2
              candidat      16
   election_territoire    1073
resultat_participation    2146
     resultat_candidat   14484
       type_indicateur       5
            indicateur      45
            prediction       0

Total : 18308 lignes


In [19]:
# Explorer communes Gironde via ORM
query_communes = session.query(
    Commune.id_commune,
    Commune.nom_commune,
    Commune.population,
    Commune.superficie_km2
).filter(
    Commune.id_departement == '33'
).order_by(
    Commune.population.desc().nullslast()
)

df_communes = pd.read_sql(query_communes.statement, session.bind)

print(f"{len(df_communes)} communes en Gironde")
print(f"\nTop 20 par population :")
print(df_communes.head(20).to_string(index=False))
print(f"\nStatistiques :")
print(df_communes[['population', 'superficie_km2']].describe())

534 communes en Gironde

Top 20 par population :
id_commune            nom_commune  population superficie_km2
     33063               Bordeaux      267991           None
     33281               Mérignac       78090           None
     33318                 Pessac       67339           None
     33522                Talence       46338           None
     33550      Villenave-d'Ornon       42545           None
     33449 Saint-Médard-en-Jalles       32910           None
     33039                 Bègles       31831           None
     33529       La Teste-de-Buch       27566           None
     33192              Gradignan       26952           None
     33119                  Cenon       26834           None
     33249                Lormont       25769           None
     33069             Le Bouscat       25081           None
     33243               Libourne       25036           None
     33162                Eysines       24825           None
     33199          Gujan-Mestras   

In [ ]:
# Distribution population + superficie
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme population (log scale)
pop = df_communes['population'].dropna()
ax1.hist(pop, bins=50, color='steelblue', edgecolor='white', alpha=0.8)
ax1.set_xscale('log')
ax1.set_xlabel('Population (echelle log)')
ax1.set_ylabel('Nombre de communes')
ax1.set_title('Distribution de la population des communes (Gironde)')
ax1.axvline(pop.median(), color='red', linestyle='--', label=f'Mediane: {pop.median():.0f}')
ax1.legend()

# Boxplot superficie
sup = df_communes['superficie_km2'].dropna().astype(float)
if len(sup) > 0:
    ax2.boxplot(sup, vert=True)
    ax2.set_ylabel('Superficie (km2)')
    ax2.set_title('Distribution superficie des communes (Gironde)')
else:
    ax2.text(0.5, 0.5, 'Superficie non disponible', ha='center', va='center', transform=ax2.transAxes)
    ax2.set_title('Superficie (donnees absentes)')

plt.tight_layout()
plt.savefig('../docs/figures/exploration/distribution_communes.png', dpi=150, bbox_inches='tight')
plt.show()

In [21]:
# Charger participation via ORM (JOIN Election)
# Note : id_territoire format '33XXXXX' -> on normalise en code INSEE '3XXXX' via [2:]
query_participation = session.query(
    ResultatParticipation.id_territoire,
    ResultatParticipation.tour,
    ResultatParticipation.nombre_inscrits,
    ResultatParticipation.nombre_votants,
    ResultatParticipation.nombre_exprimes,
    ResultatParticipation.nombre_abstentions,
    ResultatParticipation.nombre_blancs_nuls,
    ResultatParticipation.pourcentage_votants,
    ResultatParticipation.pourcentage_abstentions,
    Election.annee
).join(
    Election, ResultatParticipation.id_election == Election.id_election
).filter(
    ResultatParticipation.type_territoire == 'COMMUNE'
)

df_participation = pd.read_sql(query_participation.statement, session.bind)

# Normaliser id_territoire -> code INSEE commune
df_participation['commune_id'] = df_participation['id_territoire'].str[2:]

# Joindre infos commune en pandas
df_participation = df_participation.merge(
    df_communes[['id_commune', 'nom_commune', 'population']],
    left_on='commune_id', right_on='id_commune', how='left'
)

print(f"{len(df_participation)} lignes de participation")
print(f"Annees : {sorted(df_participation['annee'].unique())}")
print(f"Tours  : {sorted(df_participation['tour'].unique())}")
df_participation.head()

2146 lignes de participation
Annees : [np.int64(2017), np.int64(2022)]
Tours  : [np.int64(1), np.int64(2)]


,id_territoire,tour,nombre_inscrits,nombre_votants,nombre_exprimes,nombre_abstentions,nombre_blancs_nuls,pourcentage_votants,pourcentage_abstentions,annee,commune_id,id_commune,nom_commune,population
0,3333001,1,1477,1106,1078,371,28,74.88,25.12,2017,33001,33001,Abzac,2048.0
1,3333002,1,598,502,481,96,21,83.95,16.05,2017,33002,33002,Aillas,786.0
2,3333003,1,9965,7712,7534,2253,178,77.39,22.61,2017,33003,33003,Ambarès-et-Lagrave,17644.0
3,3333004,1,2210,1624,1582,586,42,73.48,26.52,2017,33004,33004,Ambès,3313.0
4,3333005,1,10771,8833,8678,1938,155,82.01,17.99,2017,33005,33005,Andernos-les-Bains,12710.0


In [ ]:
# Visualiser participation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Barplot taux participation moyen par (annee, tour)
part_moy = df_participation.groupby(['annee', 'tour'])['pourcentage_votants'].mean().reset_index()
part_moy['label'] = part_moy['annee'].astype(str) + ' T' + part_moy['tour'].astype(str)
colors = ['steelblue', 'lightblue', 'coral', 'lightsalmon']
bars = ax1.bar(part_moy['label'], part_moy['pourcentage_votants'].astype(float), color=colors[:len(part_moy)])
for bar in bars:
    h = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., h + 0.5, f'{h:.1f}%', ha='center', fontweight='bold')
ax1.set_ylabel('Taux de participation moyen (%)')
ax1.set_title('Participation moyenne par election (534 communes)')
ax1.set_ylim(0, 100)

# Scatter participation vs population
t1_2022 = df_participation[(df_participation['annee'] == 2022) & (df_participation['tour'] == 1)]
ax2.scatter(t1_2022['population'], t1_2022['pourcentage_votants'].astype(float), alpha=0.4, s=15, c='coral')
ax2.set_xscale('log')
ax2.set_xlabel('Population (log)')
ax2.set_ylabel('Taux de participation (%)')
ax2.set_title('Participation vs Population - 2022 T1')

plt.tight_layout()
plt.savefig('../docs/figures/exploration/participation.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Charger resultats candidats via ORM (JOIN Election + Candidat)
query_resultats = session.query(
    ResultatCandidat.id_territoire,
    ResultatCandidat.tour,
    ResultatCandidat.nombre_voix,
    ResultatCandidat.pourcentage_voix_exprimes,
    Election.annee,
    Candidat.nom,
    Candidat.prenom
).join(
    Election, ResultatCandidat.id_election == Election.id_election
).join(
    Candidat, ResultatCandidat.id_candidat == Candidat.id_candidat
).filter(
    ResultatCandidat.type_territoire == 'COMMUNE'
)

df_resultats = pd.read_sql(query_resultats.statement, session.bind)

# Normaliser id_territoire -> code INSEE
df_resultats['commune_id'] = df_resultats['id_territoire'].str[2:]
df_resultats['candidat_nom'] = df_resultats['prenom'] + ' ' + df_resultats['nom']
# NULL pourcentage = 0 voix = 0%
df_resultats['pct'] = df_resultats['pourcentage_voix_exprimes'].fillna(0).astype(float)

# Joindre infos commune
df_resultats = df_resultats.merge(
    df_communes[['id_commune', 'nom_commune', 'population']],
    left_on='commune_id', right_on='id_commune', how='left'
)

print(f"{len(df_resultats)} lignes de resultats candidats")
print(f"Candidats uniques : {df_resultats['candidat_nom'].nunique()}")
print(f"Communes          : {df_resultats['commune_id'].nunique()}")

In [ ]:
# Top candidats T1 par annee - barplot horizontal
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

for ax, annee, color in [(ax1, 2017, 'steelblue'), (ax2, 2022, 'coral')]:
    t1 = df_resultats[(df_resultats['annee'] == annee) & (df_resultats['tour'] == 1)]
    moy = t1.groupby('candidat_nom')['pct'].mean().sort_values(ascending=True)
    if len(moy) > 0:
        moy.plot(kind='barh', ax=ax, color=color)
        for i, (v, name) in enumerate(zip(moy.values, moy.index)):
            ax.text(v + 0.3, i, f'{v:.1f}%', va='center', fontsize=8)
    ax.set_xlabel('% voix exprimes (moyenne communes)')
    ax.set_title(f'Presidentielles {annee} - Tour 1')

plt.tight_layout()
plt.savefig('../docs/figures/exploration/top_candidats_t1.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Comparaison 2017 vs 2022 - 7 candidats communs
candidats_2017 = set(df_resultats[(df_resultats['annee'] == 2017) & (df_resultats['tour'] == 1)]['candidat_nom'].unique())
candidats_2022 = set(df_resultats[(df_resultats['annee'] == 2022) & (df_resultats['tour'] == 1)]['candidat_nom'].unique())
candidats_communs = sorted(candidats_2017 & candidats_2022)

print(f"{len(candidats_communs)} candidats communs 2017-2022 :")
for c in candidats_communs:
    print(f"  - {c}")

# Barplot compare cote a cote
t1 = df_resultats[df_resultats['tour'] == 1]
t1_communs = t1[t1['candidat_nom'].isin(candidats_communs)]
moy_pivot = t1_communs.groupby(['annee', 'candidat_nom'])['pct'].mean().unstack('annee')
moy_pivot = moy_pivot.sort_values(2022, ascending=True)

fig, ax = plt.subplots(figsize=(12, 6))
moy_pivot.plot(kind='barh', ax=ax, color=['steelblue', 'coral'])
ax.set_xlabel('% voix exprimes (moyenne communes)')
ax.set_title('Comparaison 2017 vs 2022 - 7 candidats communs (T1)')
ax.legend(title='Annee')

plt.tight_layout()
plt.savefig('../docs/figures/exploration/comparaison_2017_2022.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Heatmap communes x candidats (top 10 communes par population)
top10_communes = df_communes.head(10)['id_commune'].tolist()

# T1 2022 pour les communes top 10
t1_2022_top = df_resultats[
    (df_resultats['annee'] == 2022) &
    (df_resultats['tour'] == 1) &
    (df_resultats['commune_id'].isin(top10_communes))
]

# Pivot : communes en lignes, candidats en colonnes
heatmap_data = t1_2022_top.pivot_table(
    index='nom_commune', columns='candidat_nom', values='pct', aggfunc='first'
)

# Garder les 6 candidats avec le plus de voix
top_cands = heatmap_data.mean().nlargest(6).index
heatmap_data = heatmap_data[top_cands]

fig, ax = plt.subplots(figsize=(14, 7))
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='YlOrRd', ax=ax, linewidths=0.5)
ax.set_title('% voix exprimes - 2022 T1 (top 10 communes x top 6 candidats)')
ax.set_ylabel('')

plt.tight_layout()
plt.savefig('../docs/figures/exploration/heatmap_communes_candidats.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Indicateurs securite Bordeaux via ORM (JOIN TypeIndicateur)
query_indicateurs = session.query(
    Indicateur.annee,
    Indicateur.valeur_numerique,
    TypeIndicateur.code_type,
    TypeIndicateur.nom_affichage
).join(
    TypeIndicateur, Indicateur.id_type == TypeIndicateur.id_type
).filter(
    Indicateur.id_territoire == '33063',
    Indicateur.type_territoire == 'COMMUNE'
).order_by(
    Indicateur.annee, TypeIndicateur.code_type
)

df_indicateurs = pd.read_sql(query_indicateurs.statement, session.bind)
df_indicateurs['valeur'] = df_indicateurs['valeur_numerique'].astype(float)

print(f"{len(df_indicateurs)} indicateurs securite (Bordeaux, 2016-2024)")
print(f"Types : {df_indicateurs['code_type'].unique().tolist()}")

# Lineplot evolution 2016-2024
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Criminalite totale seule (echelle differente)
crim = df_indicateurs[df_indicateurs['code_type'] == 'CRIMINALITE_TOTALE'].sort_values('annee')
ax1.plot(crim['annee'], crim['valeur'], marker='o', linewidth=2, color='darkred', markersize=8)
for x, y in zip(crim['annee'], crim['valeur']):
    ax1.text(x, y + 400, f'{int(y):,}', ha='center', fontsize=8)
ax1.axvline(2017, color='blue', ls='--', alpha=0.4, label='Election 2017')
ax1.axvline(2022, color='orange', ls='--', alpha=0.4, label='Election 2022')
ax1.set_title('Criminalite Totale - Bordeaux (2016-2024)')
ax1.set_ylabel('Nombre de faits')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Autres indicateurs
for code in df_indicateurs['code_type'].unique():
    if code != 'CRIMINALITE_TOTALE':
        sub = df_indicateurs[df_indicateurs['code_type'] == code].sort_values('annee')
        ax2.plot(sub['annee'], sub['valeur'], marker='o', linewidth=2, label=code)

ax2.set_title('Indicateurs Securite detailles - Bordeaux (2016-2024)')
ax2.set_ylabel('Nombre de faits')
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../docs/figures/exploration/indicateurs_securite.png', dpi=150, bbox_inches='tight')
plt.show()

## Conclusions

**Donnees validees :**
- 534 communes Gironde avec population et superficie
- Participation complete pour 2017 & 2022 (T1 + T2)
- 14 484 resultats candidats (multi-communes, multi-tours)
- 45 indicateurs securite Bordeaux (5 types x 9 ans)

**Observations :**
- Macron et Melenchon progressent entre 2017 et 2022
- Le Pen presente dans les deux elections mais faible en Gironde
- Forte heterogeneite de participation entre communes rurales et urbaines
- Criminalite en hausse tendancielle (sauf 2020 COVID)

**7 candidats communs 2017-2022 :** Macron, Le Pen, Melenchon, Arthaud, Dupont-Aignan, Lassalle, Poutou

**Prochaine etape :** Notebook 02 - Feature Engineering + ML (Random Forest)